In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import sys
import time
sys.path.append("../Pytorch-Quaternion-Neural-Networks/core_qnn")
from quaternion_layers import *
import quaternion_ops
import functions
from layers import *
import convert_to_quaternion
from matplotlib import pyplot as plt
import quaternion
from pyquaternion import Quaternion
import importlib
from quaternion import QuaternionTensor

In [2]:
# our quaternion NN


class OurNet(nn.Module):
    def __init__(self):
        super(OurNet, self).__init__()
        self.conv1 = QConv2d(4, 8, 5, 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = QConv2d(8, 16, 5, 1)
        self.bn = QBatchNorm2d(16)
        self.fc1 = QLinear(16*5*5, 120)
        self.fc2 = QLinear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.elu(self.conv1(x)))
        x = self.pool(F.elu(self.bn(self.conv2(x))))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x    
    
# standard real-valued CNN
class RealNet(nn.Module):
    def __init__(self):
        super(RealNet, self).__init__()
        self.conv1 = nn.Conv2d(4, 8, 5, 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(8, 16, 5, 1)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Parcollet's CNN
class ParcolletNet(nn.Module):
    def __init__(self):
        super(ParcolletNet, self).__init__()
        self.conv1 = QuaternionConv(4, 8, 5, 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = QuaternionConv(8, 16, 5, 1)
        self.fc1 = QuaternionLinear(16*5*5, 120)
        self.fc2 = QuaternionLinear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def train(model):
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    for epoch in range(2):  # loop over the dataset multiple times

        running_loss = 0.0
        start = time.time()
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            gray_image = grayscale(inputs)
            # inputs' channels must be divisible by 4 in a QNN,
            # here used to compare results
            inputs = torch.cat([inputs, gray_image],1)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs.to(device))
            loss = criterion(outputs, labels.to(device))
            loss.backward()
            optimizer_real.step()

            # print statistics
            running_loss += loss_real.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0

                print(time.time() - start)

    print('Finished Training')

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
torch.manual_seed(0)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64,
                                          shuffle=True, num_workers=0)

grayscale = transforms.Grayscale(num_output_channels=1)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified


In [4]:
# run on gpu if it is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
ournet = OurNet().to(device)
parcnet = ParcolletNet().to(device)
realnet = RealNet().to(device)

In [5]:
def train(model):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    for epoch in range(1):  # loop over the dataset multiple times

        running_loss = 0.0
        start = time.time()
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            gray_image = grayscale(inputs)
            # inputs' channels must be divisible by 4 in a QNN,
            # here used to compare results
            inputs = torch.cat([inputs, gray_image],1)
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs.to(device))
            loss = criterion(outputs, labels.to(device))
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            if i % 200 == 199:    
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.0
                print("time elapsed: ", time.time() - start)
                
    print('Finished Training')

In [6]:
train(ournet)

tensor(1.0490, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBa

tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackw

tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackw

tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackward0>)
tensor(nan, device='cuda:0', grad_fn=<VarBackw

KeyboardInterrupt: 

In [ ]:
train(ournet)

In [ ]:
train(parcnet)

In [ ]:
train(realnet)

In [ ]:
ournet.bn.running_invsq_cov

In [ ]:
x = torch.randn(16, 4, 32, 32)
bn = QBatchNorm2d(4, affine=False)

In [7]:
17/12

1.4166666666666667

In [ ]:
axes, d = (1, *range(3, x.dim())), x.shape[0]
shape = 1, x.shape[2], *([1] * (x.dim() - 3))
mean = x.mean(dim=axes)
x = x - mean.reshape(x.shape[0], *shape)
perm = x.permute(2, 0, *axes).flatten(2, -1)
cov = torch.matmul(perm, perm.transpose(-1, -2)) / perm.shape[-1]
ell = torch.cholesky(cov, upper=True)

In [ ]:
%timeit soln = torch.triangular_solve(
    x.unsqueeze(-1).permute(*range(1, x.dim()), 0, -1),
    ell.reshape(*shape, d, d)
)

invsq_cov = soln.solution.squeeze(-1)
z = torch.stack(torch.unbind(invsq_cov, dim=-1), dim=0)

In [ ]:
invsq_cov.unsqueeze(4)

In [ ]:
ell.inverse().shape

In [ ]:
%timeit torch.matmul(ell.inverse().reshape(*shape, d, d), x.unsqueeze(-1).permute(*range(1, x.dim()), 0, -1))